# What neighborhood in  NYC is the best location to open a gym?

## Problem Statement:  Gyms are becoming more and more like restaurants.  There is one now on every corner.  And like restaturants many go out of business.  So, how can we find the best neighborhood that would have the best chance of success.

## Background:  There  are 4 things to consider when choosing the best location for your brand new gym. The location that is going to get you the most clients and the most bang for your buck.

* Price 
* Area Demographics
* Competing Gyms
* Gym Accessibilty

## Price - the first thing you’ll want to consider is the price of the space you are trying to rent, or the land you would need to buy and develop. It’s easy to get lost in big dreams and amazing acoustics but if the funds aren’t there you won’t get far.
## What is the real estate market like?


## Area Demographics - The neigborhood your gym is in will make or break you.  Do you want an upscale neighborhood?  Do you want a neighborhood close to major shopping to bring in people.  Maybe coffe shops and lunch places for the business crowd.  We need to determine what neighborhoodbest has that.

## Competing Gyms -  As mentioned above this is possibly the biggest question to ask.  How many gyms and maybe more importantly what types of gyms are in the neighorhood you want.   A complementary workouts area can be good.  For example, many people like doing yoga and hard intense workouts.  Being near a yoga or pilates place might draw business.

## Gym accessibility - The big question here is -  What is the parking situation like.  The neighborhood where the gym is going must have ample parking.  Not necesarily a deal breaker as I've seen gyms succeed with little to no parking,  but something to consider for the perfect spot.

# Data Collection
## I will be using New York City Neighborhood Names point file that was created as a guide to New York City’s neighborhoods that appear on the web resource, “New York: A City of Neighborhoods.
## FourSquare will also be used to obtain appropriate information for each neighborhood